In [1]:
# Written by Eric Martin for COMP9021


'''
A Queue abstract data type
'''


class EmptyQueueError(Exception):
    def __init__(self, message):
        self.message = message


class Queue:
    min_capacity = 10

    def __init__(self, capacity = min_capacity):
        self.min_capacity = capacity
        self._data = [None] * capacity
        self._length = 0
        self._front = 0
        
    def __len__(self):
        '''
        >>> len(Queue(100))
        0
        '''
        return self._length

    def is_empty(self):
        return self._length == 0

    def peek_at_front(self):
        '''
        >>> queue = Queue()
        >>> queue.peek_at_front()
        Traceback (most recent call last):
        ...
        EmptyQueueError: Cannot peek at front of empty queue
        '''
        if self.is_empty():
            raise EmptyQueueError('Cannot peek at front of empty queue')
        return self._data[self._front]

    def peek_at_back(self):
        '''
        >>> queue = Queue()
        >>> queue.peek_at_back()
        Traceback (most recent call last):
        ...
        EmptyQueueError: Cannot peek at back of empty queue
        '''
        if self.is_empty():
            raise EmptyQueueError('Cannot peek at back of empty queue')
        return self._data[(self._front + self._length - 1) % len(self._data)]

    def enqueue(self, datum):
        '''
        >>> queue = Queue(1)
        >>> queue.enqueue(0)
        >>> queue.peek_at_back()
        0
        >>> print(len(queue._data))
        1
        >>> queue.enqueue(1)
        >>> queue.peek_at_back()
        1
        >>> print(len(queue._data))
        2
        >>> queue.enqueue(2)
        >>> queue.peek_at_back()
        2
        >>> print(len(queue._data))
        4
        >>> queue.enqueue(3)
        >>> queue.peek_at_back()
        3
        >>> print(len(queue._data))
        4
        >>> queue.enqueue(4)
        >>> queue.peek_at_back()
        4
        >>> print(len(queue._data))
        8
        '''
        if self._length == len(self._data):
            self._resize(2 * len(self._data))
        self._data[(self._front + self._length) % len(self._data)] = datum
        self._length += 1

    def dequeue(self):
        '''
        >>> queue = Queue(4)
        >>> for i in range(8, -1, -1): queue.enqueue(i)
        >>> print(len(queue._data))
        16
        >>> print(queue.dequeue())
        8
        >>> print(len(queue._data))
        16
        >>> print(queue.dequeue())
        7
        >>> print(len(queue._data))
        16
        >>> print(queue.dequeue())
        6
        >>> print(len(queue._data))
        16
        >>> print(queue.dequeue())
        5
        >>> print(len(queue._data))
        16
        >>> print(queue.dequeue())
        4
        >>> print(len(queue._data))
        8
        >>> print(queue.dequeue())
        3
        >>> print(len(queue._data))
        8
        >>> print(queue.dequeue())
        2
        >>> print(len(queue._data))
        4
        >>> print(queue.dequeue())
        1
        >>> print(len(queue._data))
        4
        >>> print(queue.dequeue())
        0
        >>> print(len(queue._data))
        4
        >>> print(queue.dequeue())
        Traceback (most recent call last):
        ...
        EmptyQueueError: Cannot dequeue from empty queue
        '''
        if self.is_empty():
            raise EmptyQueueError('Cannot dequeue from empty queue')
        datum_at_front = self._data[self._front]
        # Not necessary, only done to possibly hasten garbage collection
        # of element being removed from the deque.
        self._data[self._front] = None
        self._front = (self._front + 1) % len(self._data)        
        self._length -= 1
        self._shrink_if_needed()
        return datum_at_front

    def _resize(self, new_size):
        # In any case, the element at position self._front will be at position 0 in new list.
        end = self._front + new_size
        # We are shrinking to a smaller list, and not wrapping in original list.
        if end <= len(self._data):
            self._data = self._data[self._front: end]
        # We are shrinking to a smaller list, but wrapping in original list.
        elif new_size <= len(self._data):
            # There are len(self._data) - self._front data in self._data[self._front: ],
            # and new_size - (len(self._data) - self._front) == end - len(self._data).
            self._data = self._data[self._front: ] + self._data[: end - len(self._data)]
        # We are expanding to a larger list.
        else:
            # The first two lists have a total length of len(self._data).
            self._data = (self._data[self._front: ] + self._data[: self._front] +
                          [None] * (new_size - len(self._data)))
        self._front = 0

    def _shrink_if_needed(self):
        # When the queue is one quarter full, we reduce its size to make it half full,
        # provided that it would not reduce its capacity to less than the minimum required.
        if self.min_capacity // 2 <= self._length <= len(self._data) // 4:
            self._resize(len(self._data) // 2)


if __name__ == '__main__':
    import doctest
    doctest.testmod()    

        

In [76]:
import sys
from random import seed, choice
from queue_adt import *

def display_grid():
    for row in grid:
        print('    ', *row)

def overturn_grid():
    global grid
    grid = list(map(list,zip(*grid)))
        
    
def preferred_paths_to_corners():
    overturn_grid()
    TruthTable = [[True for i in range(dim)] for j in range(dim)]
    record = [[(i,j) for j in range(dim)] for i in range(dim)]
#     record = list(map(list,zip(*record)))
    paths = dict([])
    queue = Queue()
    dict_directions = dict ([])
    dict_directions['N'], dict_directions['S'], dict_directions['W'], dict_directions['E'] = [0,-1], [0,1], [-1,0], [1,0]
    queue.enqueue([3,3])
    TruthTable[3][3] = False
    while not queue.is_empty():
        (x,y) = queue.dequeue()
        if (x,y) not in corners:
            w1 = grid[x][y][0]
            w2 = grid[x][y][1]
            for t in [[x+dict_directions[w1][0]+dict_directions[w2][0], y+dict_directions[w1][1]+dict_directions[w2][1]],[x+dict_directions[w1][0], y+dict_directions[w1][1]], [x+dict_directions[w2][0], y+dict_directions[w2][1]]]:
                if (0 <= t[0] < dim) and (0 <= t[1] < dim):
                    if TruthTable[t[0]][t[1]]:
                        queue.enqueue(t)
                        TruthTable[t[0]][t[1]] = False
                        record[t[0]][t[1]] = (x, y)
     
    for point in corners:
        if not TruthTable[point[0]][point[1]]:
            paths[point] = [point]
            node = point
            while node != (size,size):
                paths[point] = [record[node[0]][node[1]]] + paths[point]
                node = paths[point][0]
        
    return paths
    
  


try:
    seed_arg = int(input('Enter an integer: '))
except ValueError:
    print('Incorrect input, giving up.')
    sys.exit()
    
seed(seed_arg)
size = 3
dim = 2 * size + 1
grid = [[0] * dim for _ in range(dim)]
directions = 'NE', 'SE', 'SW', 'NW'

grid = [[choice(directions) for _ in range(dim)] for _ in range(dim)]
print('Here is the grid that has been generated:')
display_grid()

corners = (0, 0), (dim - 1, 0), (dim - 1, dim - 1), (0, dim - 1)
paths = preferred_paths_to_corners()
if not paths:
    print('There is no path to any corner')
    sys.exit()
for corner in corners:
    if corner not in paths:
        print(f'There is no path to {corner}')
    else:
        print(f'The preferred path to {corner} is:')
        print('  ', paths[corner])

Enter an integer: 5
Here is the grid that has been generated:
     SW SW NE NW SE NE SE
     NE SW NW SE NW NE SE
     NE SE NW SW SE NW SE
     NE SE NW SE SE NE NE
     SE SE SE SE SW SW SE
     SE SE SE NW SW NE SW
     NW SE SE SW NE SW SW
The preferred path to (0, 0) is:
   [(3, 3), (4, 3), (5, 3), (5, 2), (4, 1), (3, 1), (3, 2), (2, 2), (1, 1), (0, 1), (0, 0)]
The preferred path to (6, 0) is:
   [(3, 3), (4, 3), (5, 3), (5, 2), (5, 1), (6, 0)]
The preferred path to (6, 6) is:
   [(3, 3), (4, 3), (5, 4), (5, 5), (6, 5), (6, 6)]
There is no path to (0, 6)


In [19]:
L = [[1,2],[3,4]]
print(L)
L = list(map(list,zip(*L)))
print(L)

[[1, 2], [3, 4]]
[[1, 3], [2, 4]]


In [39]:
preferred_paths_to_corners()


[[False, False, False, False, False, False, False], [False, False, False, False, False, False, False], [False, False, False, False, False, False, False], [False, False, False, False, False, False, False], [False, False, False, False, False, False, False], [False, False, False, False, False, False, False], [False, False, False, False, False, False, False]]


In [43]:
# Randomly fills a grid of size 7 x 7 with NE, SE, SW, NW,
# meant to represent North-East, South-East, North-West, South-West,respectively,
# and starting from the cell in the middle of the grid,
# determines, for each of the 4 corners of the grid, the preferred path amongst
# the shortest paths that reach that corner, if any. At a given cell, it is possible to move
# according to any of the 3 directions indicated by the value of the cell;
# e.g., from a cell storing NE, it is possible to move North-East, East, or North.
# At any given point, one prefers to move diagonally, then horizontally,
# and vertically as a last resort.
# plz watch Lecture Video at 2:22 2nd OCT
# BFS is optimal when actions are unweighted, BFS can give you the shortest path (optimal shortest)
# need queue_adt for BFS
# 输出坐标系是 (j,i), j是竖着column的index,  i是横着行的index, preferred_paths_to_corners()输出前需处理一下
# 左上角坐标 (0,0) j=0 and i=0
# 左下角坐标 (0,6) j=0 and i=6
# 右上角坐标 (6,0) j=6 and i=0
# 右下角坐标 (6,6) j=6 and i=6

import sys
from random import seed, choice
from queue_adt import *

def display_grid():
    for i in range(dim):
        print('', end = '')
        for j in range(dim):
            print(' ', grid[i][j], end = '')
        print()
    print()

# Using BFS to find optimize path, diagonal > horizontal > vertical
def BFS():
    paths, q , di = dict([]), Queue(), dict([])
    q.enqueue([3,3])
    visited, visited[3][3] = [[False for i in range(dim)] for j in range(dim)], True
    pre = [[(i,j) for i in range(dim)] for j in range(dim)] # pre to record previous node
    di['N'], di['S'], di['W'], di['E'] = [-1, 0], [1, 0], [0, -1], [0, 1]
    while not q.is_empty():
        r, c = q.dequeue()
        # Stop once reach any corner, Otherwise keep BFS
        if (r,c) not in corners:
            d1, d2 = grid[r][c][0], grid[r][c][1]
            for t in [[r+di[d1][0]+di[d2][0], c+di[d1][1]+di[d2][1]], [r+di[d1][0], c+di[d1][1]], [r+di[d2][0], c+di[d2][1]]]:
                if (t[0] >=0 and t[0] < dim) and (t[1] >= 0 and t[1] < dim):
                    if not visited[t[0]][t[1]]:
                        visited[t[0]][t[1]] = True
                        print(t)
                        q.enqueue(t)
                        pre[t[0]][t[1]] = (r, c)
    for c in corners:
        key = (c[1], c[0])
        if visited[c[0]][c[1]]:
            paths[key], tmp = [c] , c
            while tmp != (size, size):
                paths[key] = [pre[tmp[0]][tmp[1]]] + paths[key]
                tmp = paths[key][0]
    
    return paths

def preferred_paths_to_corners():
    paths = BFS()
    for k in paths.keys(): # need adjust ouput coordinate
        for i in range(len(paths[k])):
            paths[k][i] = (paths[k][i][1], paths[k][i][0])
    return paths

try:
    seed_arg = int(input('Enter an integer: '))
except ValueError:
    print('Incorrect input, giving up.')
    sys.exit()

seed(seed_arg)
size = 3
dim = 2 * size + 1
grid = [[0] * dim for _ in range(dim)]
directions = 'NE', 'SE', 'SW', 'NW'

for i in range(dim):
    for j in range(dim):
        grid[i][j] = choice(directions)
print('Here is the grid that has been generated:')
display_grid()

corners = (0, 0), (dim - 1, 0), (dim - 1, dim - 1), (0, dim - 1)
paths = preferred_paths_to_corners()
if not paths:
    print('There is no path to any corner')
    sys.exit()
for corner in corners:
    if corner not in paths:
        print('There is no path to {}'.format(corner))
    else:
        print('The preferred path to {} is:'.format(corner))
        print('  ', paths[corner])

'''
上课朝朝心累,编程夜不能寐,
作业太多不会,代写收费嫌贵,
查重分数作废,千杯独饮不醉,
马丁回复干脆，俩字 ”劝退!”

__init__杰克买买提 6 Oct 😔
Wechat: specialjack2
'''


Enter an integer: 1
Here is the grid that has been generated:
  SE  NE  SW  NE  NW  NW  NW
  NW  SE  NE  NW  NE  NW  NW
  NE  NW  SW  SE  NE  SW  NE
  NE  NE  NE  NW  SE  NW  NE
  SE  NW  NW  SE  SW  SE  SE
  NW  SW  NE  NW  NE  SE  SW
  NE  SW  NW  SE  SW  SW  NW

[2, 2]
[2, 3]
[3, 2]
[3, 1]
[2, 1]
[3, 4]
[2, 4]
[1, 0]
[1, 1]
[2, 0]
[4, 5]
[4, 4]
[3, 5]
[1, 5]
[1, 4]
[2, 5]
[0, 0]
[1, 2]
[5, 6]
[5, 5]
[4, 6]
[5, 3]
[5, 4]
[4, 3]
[0, 4]
[0, 5]
[0, 3]
[0, 2]
[1, 3]
[6, 5]
[6, 6]
[4, 2]
[5, 2]
[0, 1]
[6, 4]
[4, 1]
[6, 3]
[3, 0]
[4, 0]
[5, 1]
[5, 0]
[6, 0]
[6, 1]
The preferred path to (0, 0) is:
   [(3, 3), (2, 2), (1, 2), (0, 1), (0, 0)]
There is no path to (6, 0)
The preferred path to (6, 6) is:
   [(3, 3), (3, 2), (4, 3), (5, 4), (6, 5), (6, 6)]
The preferred path to (0, 6) is:
   [(3, 3), (3, 2), (4, 3), (4, 4), (3, 5), (2, 4), (1, 4), (0, 4), (1, 5), (0, 6)]


'\n上课朝朝心累,编程夜不能寐,\n作业太多不会,代写收费嫌贵,\n查重分数作废,千杯独饮不醉,\n马丁回复干脆，俩字 ”劝退!”\n\n__init__杰克买买提 6 Oct 😔\nWechat: specialjack2\n'